## Calibration

The `Calibration` class provides a way to adjust weights of observations in a dataset to match specified target values. This is commonly used in survey research and policy modeling for rebalancing datasets to better represent desired population characteristics. 

The calibration process uses an optimization algorithm to find weights that minimize the distance from the original weights while achieving the target constraints.

## Basic usage

### Parameters

`__init__(data, weights, targets)`

- `data` (pd.DataFrame): The dataset to be calibrated. This should contain all the variables you want to use for calibration.
- `weights` (np.ndarray): Initial weights for each observation in the dataset. Typically starts as an array of ones for equal weighting.
- `targets` (np.ndarray): Target values that the calibration process should achieve. These correspond to the desired weighted sums.

Calibration can be easily done by initializing the `Calibration` class, passing in the parameters above. Then `calibrate()` method performs the actual calibration using the reweight function. This method:
- Adjusts the weights to better match the target values
- May subsample the data for efficiency
- Updates both `self.weights` and `self.data` with the calibrated results

## Example

Below is a complete example showing how to calibrate a dataset to match income targets for specific age groups:

In [8]:
from microcalibrate.calibration import Calibration
import logging
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

logging.basicConfig(
    level=logging.INFO,
)

# Create a sample dataset with age and income data
random_generator = np.random.default_rng(0)
data = pd.DataFrame({
    "age": random_generator.integers(18, 70, size=100),
    "income": random_generator.normal(40000, 10000, size=100),
})

# Set initial weights (all one in this example)
weights = np.ones(len(data))

# Calculate target values: total income for age groups 20-30 and 40-50 (as an example) or employ existing targets
targets_matrix = pd.DataFrame({
    "income_aged_20_30": ((data["age"] >= 20) & (data["age"] <= 30)).astype(float) * data["income"],
    "income_aged_40_50": ((data["age"] >= 40) & (data["age"] <= 50)).astype(float) * data["income"],
    "income_aged_68_70" : ((data["age"] >= 68) & (data["age"] <= 70)).astype(float) * data["income"],
})

# 15% higher than the sum of data with the original weights
targets = np.array([
    (targets_matrix["income_aged_20_30"] * weights * 1.15).sum(), 
    (targets_matrix["income_aged_40_50"] * weights * 1.15).sum(), 
    (targets_matrix["income_aged_68_70"] * weights * 1.15).sum()
])

print(f"Original weights: {weights}")
print(f"Original targets: {targets}")

Original weights: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Original targets: [490669.21576586 983592.61957571 114703.42450495]


In [9]:
# Initialize the Calibration object
calibrator = Calibration(
    loss_matrix=targets_matrix,
    weights=weights, 
    targets=targets,
    noise_level=0.05,
    epochs=528,
    learning_rate=0.01,
    dropout_rate=0,
    subsample_every=0,
)

# Perform the calibration
performance_df = calibrator.calibrate()

print(f"Original dataset size: {len(targets_matrix)}")
print(f"Calibrated dataset size: {len(calibrator.loss_matrix)}")
print(f"Number of calibrated weights: {len(calibrator.weights)}")

INFO:microcalibrate.calibration:Performing basic target assessment...
INFO:microcalibrate.reweight:Starting calibration process for targets ['income_aged_20_30' 'income_aged_40_50' 'income_aged_68_70']: [490669.21576586 983592.61957571 114703.42450495]
INFO:microcalibrate.reweight:Original weights - mean: 1.0000, std: 0.0000
INFO:microcalibrate.reweight:Initial weights after noise - mean: 1.0260, std: 0.0132
Reweighting progress:   0%|          | 0/528 [00:00<?, ?epoch/s, loss=0.0122, count_observations=100, weights_mean=1.03, weights_std=0.0132, weights_min=1]INFO:microcalibrate.reweight:Within 10% from targets: 0.00% 

Reweighting progress:   0%|          | 0/528 [00:00<?, ?epoch/s, loss=0.000922, count_observations=100, weights_mean=1.06, weights_std=0.0512, weights_min=1]INFO:microcalibrate.reweight:Within 10% from targets: 100.00% 

INFO:microcalibrate.reweight:Epoch   10: Loss = 0.000922, Change = 0.011249 (improving)
Reweighting progress:   2%|▏         | 11/528 [00:00<00:05, 10

Original dataset size: 100
Calibrated dataset size: 100
Number of calibrated weights: 100


In [10]:
# Calculate final weighted totals
final_totals = targets_matrix.mul(calibrator.weights, axis=0).sum().values

print(f"Target totals: {targets}")
print(f"Final calibrated totals: {final_totals}")
print(f"Difference: {final_totals - targets}")
print(f"Relative error: {(final_totals - targets) / targets * 100}")

Target totals: [490669.21576586 983592.61957571 114703.42450495]
Final calibrated totals: [490668.22134804 983591.62770555 114702.42598393]
Difference: [-0.99441782 -0.99187016 -0.99852101]
Relative error: [-0.00020267 -0.00010084 -0.00087052]


In [11]:
np.testing.assert_allclose(
        final_totals,
        targets,
        rtol=0.01,  # relative tolerance
        err_msg="Calibrated totals do not match target values",
    )

## Basic Calibration input assesment: warnings and errors

In [13]:
# Increase one of the targets by three orders of magnitude
targets = np.array([
    (targets_matrix["income_aged_20_30"] * weights * 1000).sum(), 
    (targets_matrix["income_aged_40_50"] * weights * 1).sum(), 
    (targets_matrix["income_aged_68_70"] * weights * 1).sum()
])

calibrator = Calibration(
    loss_matrix=targets_matrix,
    weights=weights, 
    targets=targets,
    noise_level=0.05,
    epochs=128,
    learning_rate=0.01,
    dropout_rate=0,
    subsample_every=0,
)

# Perform the calibration
calibrator.calibrate()

INFO:microcalibrate.calibration:Performing basic target assessment...
INFO:microcalibrate.reweight:Starting calibration process for targets ['income_aged_20_30' 'income_aged_40_50' 'income_aged_68_70']: [4.26668883e+08 8.55297930e+05 9.97421083e+04]
INFO:microcalibrate.reweight:Original weights - mean: 1.0000, std: 0.0000
INFO:microcalibrate.reweight:Initial weights after noise - mean: 1.0256, std: 0.0143
Reweighting progress:   0%|          | 0/128 [00:00<?, ?epoch/s, loss=0.333, count_observations=100, weights_mean=1.03, weights_std=0.0143, weights_min=1]INFO:microcalibrate.reweight:Within 10% from targets: 66.67% 

Reweighting progress:   0%|          | 0/128 [00:00<?, ?epoch/s, loss=0.333, count_observations=100, weights_mean=1.03, weights_std=0.0434, weights_min=0.978]INFO:microcalibrate.reweight:Within 10% from targets: 66.67% 

INFO:microcalibrate.reweight:Epoch   10: Loss = 0.332637, Change = 0.000497 (improving)
Reweighting progress:   0%|          | 0/128 [00:00<?, ?epoch/s, 